In [1]:
!pip install sklearn

In [2]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
import gensim
import json

In [3]:
with open("C:\\Users\\tuyen.dv\\Documents\\NLP\\NLP_tuan_3\\Data\\datasetVNTC.json", encoding = "utf-8") as json_file:
  data = json.load(json_file)

In [4]:
print(data['data'][5])
print(data['target'][5])
print(data['target_names'][5])

dân_số nhật bản giảm lần đầu_tiên trong lịch_sử chính_phủ nhật bản vừa báo_động dân_số nước này đã tụt giảm lần đầu_tiên trong lịch_sử kể từ khi nhật bắt_đầu thống_kê dân_số vào năm và đây là một dấu_hiệu xấu của một trong những xã_hội già_cỗi nhanh nhất thế_giới khảo_sát hằng năm của bộ tế cho thấy trong năm nay tử suất đã vượt sinh suất mức ngàn số_liệu trên quả là điều bất_ngờ cho các nhà nhân_khẩu_học vốn cho rằng thời_điểm sớm nhất mà dân_số nhật bắt_đầu giảm là vào năm trong khi trên thực_tế sự sụt_giảm này đã diễn ra nhanh hơn nguyên_nhân của tình_trạng trên là do sinh suất của xứ_sở phù tang quá thấp trẻ bà mẹ ngoài_ra những nguyên_nhân dẫn đến việc ngại sinh con là phụ_nữ nhật hiện có khuynh_hướng lập gia_đình muộn trong khi chỗ lại chật_hẹp và chi_phí giáo_dục cao dân_số tụt_dốc có_thể đe_dọa nghiêm_trọng đến nền kinh_tế vì nó gây ra tình_trạng thiếu_hụt lao_động thiếu_hụt nguồn thuế và gánh nặng phúc_lợi xã_hội khi tỷ_lệ người lao_động đóng thuế_giảm mạnh so với số người về 

In [5]:
def get_corpus(documents):
    corpus = []
    for i in range(len(documents)):
        doc = documents[i]
        words = doc.split(' ')
        tagged_document = gensim.models.doc2vec.TaggedDocument(words, [i])
        corpus.append(tagged_document)
    return corpus
    
train_corpus = get_corpus(data['data'][0:30000])
test_corpus = get_corpus(data['data'][30000:])

In [6]:
d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=5, epochs=40)
d2v_model.build_vocab(train_corpus)

In [7]:
d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

In [8]:
X_data_vectors = []
for x in train_corpus:
    vector = d2v_model.infer_vector(x.words)
    X_data_vectors.append(vector)
X_test_vectors = []
for x in test_corpus:
    vector = d2v_model.infer_vector(x.words)
    X_test_vectors.append(vector)

In [9]:
y_train = data['target'][0:30000]
y_test = data['target'][30000:]

In [10]:
#d2v_model.save("doc2vec.model")

In [11]:
class MultilayerPerceptron(nn.Module):

  def __init__(self, input_size,output_size):
    super(MultilayerPerceptron, self).__init__()

    # Saving the initialization parameters
    self.input_size = input_size 
    self.output_size = output_size

    # Defining model
    self.model = nn.Sequential(
        nn.Linear(self.input_size, 128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Linear(32, output_size),
    )
    
  def forward(self, x):
    output = self.model(x)
    return output

In [12]:
np.asarray(X_data_vectors).shape

(30000, 300)

In [13]:
# Instantiate the model
ann_model = MultilayerPerceptron(np.asarray(X_data_vectors).shape[1], 10)

# Define the optimizer
adam = optim.Adam(ann_model.parameters(), lr=0.1)#,weight_decay=1e-5)
# sgd = optim.SGD(model.parameters(), lr = 0.01)
# Define loss using a predefined loss function
loss_function = nn.CrossEntropyLoss()

# # Calculate how our model is doing now
# y_pred = model(x)
# loss_function(y_pred, y).item()

In [14]:
n_epoch = 200
for epoch in range(n_epoch):
  # Set the gradients to 0
  adam.zero_grad()
  # Get the model predictions
  y_pred = ann_model(torch.tensor(X_data_vectors, dtype = torch.float))
  # Get the loss

  loss = loss_function(y_pred, torch.tensor(y_train))
  # Print stats
  print(f"Epoch {epoch}: traing loss: {loss}")
  # Compute the gradients
  loss.backward()
  # Take a step to optimize the weights
  adam.step()

C:\Users\tuyen.dv\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  


Epoch 0: traing loss: 2.3155720233917236
Epoch 1: traing loss: 24.67447853088379
Epoch 2: traing loss: 5.230480194091797
Epoch 3: traing loss: 2.391493558883667
Epoch 4: traing loss: 2.214385747909546
Epoch 5: traing loss: 2.0549323558807373
Epoch 6: traing loss: 1.968221664428711
Epoch 7: traing loss: 1.9911746978759766
Epoch 8: traing loss: 2.0307421684265137
Epoch 9: traing loss: 1.9876731634140015
Epoch 10: traing loss: 1.9430855512619019
Epoch 11: traing loss: 1.9036471843719482
Epoch 12: traing loss: 1.8654310703277588
Epoch 13: traing loss: 1.8299050331115723
Epoch 14: traing loss: 1.7826948165893555
Epoch 15: traing loss: 1.736605167388916
Epoch 16: traing loss: 1.6883610486984253
Epoch 17: traing loss: 1.6590442657470703
Epoch 18: traing loss: 1.625562310218811
Epoch 19: traing loss: 1.5982879400253296
Epoch 20: traing loss: 1.5733996629714966
Epoch 21: traing loss: 1.5472300052642822
Epoch 22: traing loss: 1.5185272693634033
Epoch 23: traing loss: 1.4887983798980713
Epoch 24:

In [19]:
#torch.save(ann_model.state_dict(), "ann_d2v1.pth")

In [16]:
def evaluate(X_test, y_test):
  pred = ann_model(torch.tensor(X_test, dtype = torch.float))
  output = np.argmax(pred.tolist(), axis=1)
  count = 0
  n = X_test.shape[0]
  for i in range(n):
    if(output[i] == y_test[i]):
      count += 1
  return float(count)*100/n

In [18]:
print(evaluate(np.asarray(X_data_vectors), y_train))
print(evaluate(np.asarray(X_test_vectors), y_test))

84.59333333333333
80.31391327480713
